# Library imports, configurations

In [1]:
import os
import re
import s3fs
import pandas as pd
pd.set_option('display.max_columns', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [3]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

In [4]:
BUCKET_CLEM = "clementg/diffusion"
fs.ls(BUCKET_CLEM)

['clementg/diffusion/.keep',
 'clementg/diffusion/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv']

# Functions

In [104]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].applymap(tolower)
    return df_c

# DATA Download

In [15]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])



/tmp/ipykernel_3115/1235493063.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_3115/1235493063.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


FileNotFoundError: jplaton/diffusion/ted/dila_refOrga_admin_Etat_fr_20230505

In [16]:
with fs.open(FILE_PATH_TED_S3+"/dila_refOrga_admin_Etat_fr_20230505.json", mode="rb") as file_in:  
    orga =  pd.read_json(file_in)

In [17]:
orga

,service
0,"{'plage_ouverture': [], 'site_internet': [{'li..."
1,"{'plage_ouverture': [], 'site_internet': [{'li..."
2,"{'plage_ouverture': [], 'site_internet': [{'li..."
3,"{'plage_ouverture': [], 'site_internet': [{'li..."
4,"{'plage_ouverture': [], 'site_internet': [{'li..."
...,...
7631,"{'plage_ouverture': [], 'site_internet': [{'li..."
7632,"{'plage_ouverture': [], 'site_internet': [{'li..."
7633,"{'plage_ouverture': [], 'site_internet': [{'li..."
7634,"{'plage_ouverture': [], 'site_internet': [{'li..."


In [9]:
with fs.open(BUCKET_CLEM+"/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv", mode="rb") as file_in:
    fdp_dinum_df = pd.read_csv(file_in, sep=";",encoding='Windows-1252')

In [14]:
fdp_dinum_df.head(5)

,Fichier PDF source,Group2,Poste,Référence,Site accessible aux personnes handicapées,Intitule du service demandeur - bureau - section,Corps/grade,catégorie,Riefseep,code RenoiRH,Durée si recrutement temporaire,Group1,Date,Présentation,Missions,compétences,N+1,N+2,Group3,agents à encadrer,spécificités,HO,Adresse lieu de travail,Environnement de travail,contact,Adresse mail
0,2023-1105351 - Gestionnaire financier (H-F).pdf,Choix1,Gestionnaire financier,NaN,Oui,Direction interministérielle du numérique\r\nS...,Secrétaire Administratif(ve) (B),B,Groupe 3,SP00008249,CDD 3 ans,Choix2,01/03/2023,"\rService de la Première ministre, placé sous...",Missions : \r \rVous assurez l'instruction et...,Savoirs: \r- Maitrise des règles et processus...,Cheffe de la mission budget et achats,Directrice,encadrement non,NaN,NaN,encadrement,"20, avenue de Ségur ,75007 Paris","Sur le site de Ségur, vous avez accès à un res...","Madame TAURAND Géraldine, Cheffe du pôle budge...",NaN
1,FDP - DINUM - Adjoint au chef du departement O...,Choix1,Adjoint(e) du/de la chef(fe) de département e...,NaN,Oui,Direction Interministérielle du Numérique (DIN...,-,A,-,NaN,NaN,Choix1,NaN,La Direction interministérielle du numérique (...,"En tant qu'adjoint de département ""Opérateur d...",Savoirs\r- Très bonne culture numérique et not...,Chef du département Opérateurs de produits int...,Directrice de la DINUM,encadrement,22,Direction du département assurée en binôme ave...,0,20 Avenue de Ségur 75007 Paris,"Sur le site de Ségur, vous avez accès à un res...",NaN,\r
2,FDP - DINUM - Adjoint mission budget-achats.pdf,Choix1,Adjoint au chef(fe) de la mission budget-achat...,NaN,Oui,Direction interministérielle du numérique\rMis...,-,-,-,NaN,NaN,Choix1,NaN,La Direction interministérielle du numérique ...,Missions : \r Sous l'autorité de la (du) resp...,Savoirs: \rExcellente maîtrise des mécanismes...,Chef(fe) de la mission budget-achats,Directrice interministérielle du numérique,encadrement non,NaN,contraintes calendaires fortes liées au calend...,encadrement,"20, avenue de Ségur ,75007 Paris","Sur le site de Ségur, vous avez accès à un res...",NaN,NaN
3,FDP - DINUM - Chargé (e) de mission plan de re...,Choix1,Chargé (e) de mission FTAP et plan de relance,NaN,Oui,Direction interministérielle du numérique\rMis...,-,-,-,NaN,NaN,Choix1,NaN,La Direction interministérielle du numérique ...,Missions : \rAu sein de la mission Budget-Ach...,Savoirs: \r- Bonne connaissance des environne...,Chef(e) de la mission budget et achats,Directrice interministérielle du numérique,encadrement,0/1,NaN,encadrement,"20, avenue de Ségur ,75007 Paris","Sur le site de Ségur, vous avez accès à un res...",NaN,
4,FDP - DINUM - Chef du departement Operateur de...,Choix1,Chef du département de l’Opérateur de produits...,NaN,Oui,Direction Interministériel du Numérique (DINUM...,-,A,-,NaN,NaN,Choix1,NaN,La Direction interministérielle du numérique (...,"En tant que chef de département ""Opérateur de ...",Savoirs\r- Très bonne culture numérique et not...,Directrice de la DINUM,NaN,encadrement,45,Déplacements ponctuels à prévoir (entre 8 et 1...,0,20 Avenue de Ségur 75007 Paris,"Sur le site de Ségur, vous avez accès à un res...",candidatures-dinum@pm.gouv.fr,\r


In [105]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [123]:
offres_df.head(5)

,Organisme de rattachement,Référence,Versant,Métier,Statut du poste,Nature de l'emploi,Durée du contrat,Intitulé du poste,Localisation du poste,Lieu d'affectation,Niveau d'études,Niveau d'expérience min. requis,Date de vacance de l'emploi,Date de début de publication par défaut,Date de fin de publication par défaut,Date de première publication,Spécialisation,Télétravail,Management,Employeur,Catégorie,Temps Plein,Durée du poste,Langues,Niveaux,Lieu d'affectation (sans géolocalisation),Avis de vacances au JO,Compétences attendues,Documents à transmettre,Nature de contrat
0,Structures de coopération territoriale,O043221200865282,Fonction Publique Territoriale,Chargée / Chargé de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,Géomaticien.ne écologue - SYNDICAT MIXTE DE GE...,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
1,Structures de coopération territoriale,O043221200864991,Fonction Publique Territoriale,Chargée / Chargé de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,"Chargé.e mission ""flore/habitats naturels"" vég...",Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
2,Structures de coopération territoriale,O043221200865428,Fonction Publique Territoriale,Chargée / Chargé de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,Chargé.e de missions scientifiques et techniques,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
3,Structures de coopération territoriale,O064221200866576,Fonction Publique Territoriale,"Plombière, opératrice, chauffagiste / Plombier...",Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,Agent de maintenance des bâtiments - Communaut...,Pyrénées Atlantiques (64),Hôtel de la communauté de communes de lacq-ort...,NaN,NaN,01/03/2023,28/12/2022,27/01/2023,02/12/2022,NaN,NaN,NaN,NaN,Catégorie C (employé),Oui,NaN,NaN,NaN,Mourenx,NaN,NaN,NaN,NaN
4,Centres communaux d'Action Sociale,O032221000829833,Fonction Publique Territoriale,Éducatrice / Educateur de jeunes enfants,Vacant,Emploi ouvert uniquement aux contractuels,7 mois,Puériculteur.rice / Educateur.rice de jeunes e...,Gers (32),Mairie - 32500 FLEURANCE,NaN,NaN,09/01/2023,27/10/2022,26/12/2022,03/11/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Non,NaN,NaN,NaN,FLEURANCE,NaN,NaN,NaN,NaN


In [121]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['Métier','Intitulé du poste'])
    .pipe(remove_punctuation_from_df,['Métier','Intitulé du poste'])
    .pipe(tolower_df,['Métier','Intitulé du poste'])
)

In [124]:
offres_df_clean.head(5)

,Organisme de rattachement,Référence,Versant,Métier,Statut du poste,Nature de l'emploi,Durée du contrat,Intitulé du poste,Localisation du poste,Lieu d'affectation,Niveau d'études,Niveau d'expérience min. requis,Date de vacance de l'emploi,Date de début de publication par défaut,Date de fin de publication par défaut,Date de première publication,Spécialisation,Télétravail,Management,Employeur,Catégorie,Temps Plein,Durée du poste,Langues,Niveaux,Lieu d'affectation (sans géolocalisation),Avis de vacances au JO,Compétences attendues,Documents à transmettre,Nature de contrat
0,Structures de coopération territoriale,O043221200865282,Fonction Publique Territoriale,chargee charge de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,geomaticien ne ecologue syndicat mixte de gest...,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
1,Structures de coopération territoriale,O043221200864991,Fonction Publique Territoriale,chargee charge de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,charge e mission flore habitats naturels veget...,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
2,Structures de coopération territoriale,O043221200865428,Fonction Publique Territoriale,chargee charge de politique environnementale,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,charge e de missions scientifiques et techniques,Haute Loire (43),1 rue Adrienne de Noailles - 43230 Chavaniac-L...,NaN,NaN,01/03/2023,28/12/2022,31/01/2023,01/12/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Oui,NaN,NaN,NaN,Chavaniac-Lafayette,NaN,NaN,NaN,NaN
3,Structures de coopération territoriale,O064221200866576,Fonction Publique Territoriale,plombiere operatrice chauffagiste plombier ope...,Vacant,Emploi ouvert aux titulaires et aux contractuels,NaN,agent de maintenance des batiments communaute ...,Pyrénées Atlantiques (64),Hôtel de la communauté de communes de lacq-ort...,NaN,NaN,01/03/2023,28/12/2022,27/01/2023,02/12/2022,NaN,NaN,NaN,NaN,Catégorie C (employé),Oui,NaN,NaN,NaN,Mourenx,NaN,NaN,NaN,NaN
4,Centres communaux d'Action Sociale,O032221000829833,Fonction Publique Territoriale,educatrice educateur de jeunes enfants,Vacant,Emploi ouvert uniquement aux contractuels,7 mois,puericulteur rice educateur rice de jeunes enf...,Gers (32),Mairie - 32500 FLEURANCE,NaN,NaN,09/01/2023,27/10/2022,26/12/2022,03/11/2022,NaN,NaN,NaN,NaN,Catégorie A (cadre),Non,NaN,NaN,NaN,FLEURANCE,NaN,NaN,NaN,NaN


In [129]:
offres_df_clean[offres_df_clean['Métier'].str.contains("ux designer",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data designer",na=False)][['Métier','Intitulé du poste']].head(5)

,Métier,Intitulé du poste
136654,architecte technique,expert conception et architecture de donnees d...
136668,architecte technique,expert conception et architecture de donnees d...
136672,architecte technique,expert conception et architecture de donnees d...


In [130]:
offres_df_clean[offres_df_clean['Métier'].str.contains("data scientist",na=False)|offres_df_clean['Intitulé du poste'].str.contains("data scientist",na=False)]['Organisme de rattachement'].unique()

array(['Centre Hospitalier Universitaire de Tours',
       "Muséum National d'Histoire Naturelle",
       'Direction générale des finances publiques (DGFiP)',
       "Institut national de l'information géographique et forestière",
       "Ministère de l'éducation nationale et de la jeunesse  -  Ministère de l'enseignement supérieur et de la recherche",
       'Secrétariat Général', 'Administration Centrale MENJ - MESR',
       'Autorité des Marchés Financiers',
       "Agence Régionale de Santé Provence-Alpes-Côte-d'Azur",
       'Agence de Services et de Paiement (ASP)',
       'Structures de coopération territoriale',
       'Direction Interministérielle du Numérique (DINUM)',
       'Agence Nationale de Sécurité du Médicament et des Produits de Santé',
       'Institut de Recherche pour le Développement',
       "Ministère de l'Intérieur",
       'Direction générale des entreprises (DGE)', 'DRSD',
       'Direction des Services Judiciaires',
       "Ministère des Armées - CMG d'Arcu